In [1]:
import pandas as pd 
import numpy as np 
import re
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
import cv2
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.inspection import permutation_importance
from sklearn.metrics import classification_report

from tensorflow.keras.layers import concatenate
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.metrics import CategoricalAccuracy
from keras.regularizers import l2
from tensorflow.keras.applications.inception_v3 import InceptionV3

pd.set_option('display.max_columns', None)

2022-11-16 11:11:13.364911: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_df = pd.read_csv("data/twitter_data_train_multiclass.csv")
test_df = pd.read_csv("data/twitter_data_test_multiclass.csv")

In [3]:
faces_base = "data/new batch profile pics/"

In [4]:
# get all image pathnames from base
# list to store files
res = []
res2 = []

# Iterate directory
for path in os.listdir(faces_base):
    # check if current path is a file
    if os.path.isfile(os.path.join(faces_base, path)):
        res.append(faces_base + path)
        res2.append(path)

### Pre trained CNN: InceptionV3

In [6]:
train_img_dict2 = {}
test_img_dict2 = {}
train_img2 = []
test_img2 = []
for i in range(len(res)):
    pic = res[i]
    id_name = re.match(r"[^\/\\]+(?=\.png|\.jpg)", res2[i]).group(0)
    try:
        img = cv2.imread(pic)
        if img is None:
            print("none")
            continue
        img = cv2.resize(img, (75, 75))
        if int(id_name) in list(train_df['id']):
            train_img_dict2[int(id_name)] = img 
            train_img2.append(img)
        elif int(id_name) in list(test_df['id']):
            test_img_dict2[int(id_name)] = img
            test_img2.append(img)
        #img.close()
    except Exception as e:
        print(e)

none
none
none
none
none
none
none
none
none
none
libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
none
none
none
none
libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
none
none
libpng warning: iCCP: known incorrect sRGB profile
none
none
none
none
none


In [7]:
train_img_df2 = pd.DataFrame(train_img_dict2.items(), columns = ['id', 'img'])  
test_img_df2 = pd.DataFrame(test_img_dict2.items(), columns = ['id', 'img'])  
train_df_with_img2 = pd.merge(train_img_df2, train_df, on='id')
test_df_with_img2 = pd.merge(test_img_df2, test_df, on='id')

In [8]:
base_model = InceptionV3(input_shape = (75, 75, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False
x = Flatten()(base_model.output)
x = Dense(70, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(4, activation='softmax')(x)
model = Model(base_model.input, x)
# configure the model
model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

2022-11-16 11:12:25.023570: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
x_train2, x_val2, y_train2, y_val2 = train_test_split(np.array(train_img2) / 255.0, train_df_with_img2['account_type_multi'], test_size=0.15)

In [10]:
#es_callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=15)
start_time = time.time()
inc_history = model.fit(x_train2, y_train2, epochs=20, batch_size=50, validation_data=(x_val2, y_val2))
time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds

Epoch 1/20
161/161 [==============================] - 43s 237ms/step - loss: 1.1449 - accuracy: 0.4746 - val_loss: 1.0201 - val_accuracy: 0.5442
Epoch 2/20
161/161 [==============================] - 36s 223ms/step - loss: 1.0231 - accuracy: 0.5329 - val_loss: 1.0120 - val_accuracy: 0.5456
Epoch 3/20
161/161 [==============================] - 29s 178ms/step - loss: 0.9816 - accuracy: 0.5586 - val_loss: 0.9832 - val_accuracy: 0.5675
Epoch 4/20
161/161 [==============================] - 29s 181ms/step - loss: 0.9335 - accuracy: 0.5891 - val_loss: 0.9671 - val_accuracy: 0.5823
Epoch 5/20
161/161 [==============================] - 29s 179ms/step - loss: 0.8843 - accuracy: 0.6026 - val_loss: 0.9889 - val_accuracy: 0.5682
Epoch 6/20
161/161 [==============================] - 28s 172ms/step - loss: 0.8600 - accuracy: 0.6178 - val_loss: 0.9736 - val_accuracy: 0.5760
Epoch 7/20
161/161 [==============================] - 25s 155ms/step - loss: 0.8188 - accuracy: 0.6343 - val_loss: 0.9749 - val_ac

In [11]:
score = model.evaluate(x_train2, y_train2, verbose=0)
print(f'Train loss: {score[0]} / Train accuracy: {score[1]}')

Train loss: 0.26304110884666443 / Train accuracy: 0.9310043454170227


In [12]:
score = model.evaluate(np.array(test_img2) / 255.0, test_df_with_img2['account_type_multi'], verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 1.1885908842086792 / Test accuracy: 0.5866426229476929


In [13]:
pred = model.predict(np.array(test_img2) / 255.0)
pred = np.argmax(pred,axis=1)
print(classification_report(test_df_with_img2['account_type_multi'], pred, digits=5))

52/52 [==============================] - 11s 145ms/step
              precision    recall  f1-score   support

           0    0.43421   0.40049   0.41667       412
           1    0.47119   0.49783   0.48414       460
           2    0.94714   0.78467   0.85828       274
           3    0.64323   0.70930   0.67465       516

    accuracy                        0.58664      1662
   macro avg    0.62394   0.59807   0.60844      1662
weighted avg    0.59390   0.58664   0.58825      1662



### Self-train CNN

In [112]:
train_img_dict = {}
test_img_dict = {}
train_img = []
test_img = []
for i in range(len(res)):
    pic = res[i]
    id_name = re.match(r"[^\/\\]+(?=\.png|\.jpg)", res2[i]).group(0)
    try:
        img = cv2.imread(pic)
        if img is None:
            print("none")
            continue
        if int(id_name) in list(train_df['id']):
            train_img_dict[int(id_name)] = img 
            train_img.append(img)
        elif int(id_name) in list(test_df['id']):
            test_img_dict[int(id_name)] = img
            test_img.append(img)
        #img.close()
    except Exception as e:
        print(e)

none
none
none
none
none
none
none
none
none
none
libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
none
none
none
none
libpng warning: iCCP: profile 'icc': 'RGB ': RGB color space not permitted on grayscale PNG
none
none
libpng warning: iCCP: known incorrect sRGB profile
none
none
none
none
none


In [113]:
train_img_df = pd.DataFrame(train_img_dict.items(), columns = ['id', 'img'])  
test_img_df = pd.DataFrame(test_img_dict.items(), columns = ['id', 'img'])  
train_df_with_img = pd.merge(train_img_df, train_df, on='id')
test_df_with_img = pd.merge(test_img_df, test_df, on='id')

In [117]:
kernel_size = (3, 3)
stride_size = (1, 1)
filters = [4, 10, 24]

cnn_model = Sequential()
cnn_model.add(Input(shape=(48, 48, 3)))
for f in filters:
    cnn_model.add(Conv2D(f, kernel_size, strides=stride_size, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    cnn_model.add(Conv2D(f, kernel_size, strides=stride_size, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    cnn_model.add(BatchNormalization(axis = -1))
    cnn_model.add(MaxPooling2D(pool_size=(2, 2)))

cnn_model.add(Flatten())
cnn_model.add(Dense(100, activation='relu'))
cnn_model.add(Dropout(0.1))

cnn_model.add(Dense(100, activation='relu'))
cnn_model.add(Dropout(0.1))
cnn_model.add(Dense(100, activation='relu'))

cnn_model.add(Dense(4, activation='softmax'))
# configure the model
cnn_model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

In [115]:
x_train, x_val, y_train, y_val = train_test_split(np.array(train_img) / 255.0, train_df_with_img['account_type_multi'], test_size=0.15)

In [118]:
history = cnn_model.fit(x_train, y_train, epochs=20, batch_size=50, validation_data=(x_val, y_val))

Epoch 1/20
161/161 [==============================] - 11s 64ms/step - loss: 1.6779 - accuracy: 0.4564 - val_loss: 1.8703 - val_accuracy: 0.2608
Epoch 2/20
161/161 [==============================] - 10s 60ms/step - loss: 1.4668 - accuracy: 0.5038 - val_loss: 1.7900 - val_accuracy: 0.3484
Epoch 3/20
161/161 [==============================] - 10s 65ms/step - loss: 1.3539 - accuracy: 0.5159 - val_loss: 1.4378 - val_accuracy: 0.4473
Epoch 4/20
161/161 [==============================] - 10s 61ms/step - loss: 1.2733 - accuracy: 0.5289 - val_loss: 1.2847 - val_accuracy: 0.5004
Epoch 5/20
161/161 [==============================] - 10s 64ms/step - loss: 1.2265 - accuracy: 0.5300 - val_loss: 1.2691 - val_accuracy: 0.4869
Epoch 6/20
161/161 [==============================] - 13s 81ms/step - loss: 1.1950 - accuracy: 0.5416 - val_loss: 1.2429 - val_accuracy: 0.5018
Epoch 7/20
161/161 [==============================] - 13s 83ms/step - loss: 1.1627 - accuracy: 0.5492 - val_loss: 1.2691 - val_accuracy:

In [119]:
score = cnn_model.evaluate(x_train, y_train, verbose=0)
print(f'Train loss: {score[0]} / Train accuracy: {score[1]}')

Train loss: 1.0152924060821533 / Train accuracy: 0.6313163042068481


In [120]:
score = cnn_model.evaluate(np.array(test_img) / 255.0, test_df_with_img['account_type_multi'], verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Test loss: 1.283149242401123 / Test accuracy: 0.5354993939399719


In [121]:
pred = cnn_model.predict(np.array(test_img) / 255.0)
pred = np.argmax(pred,axis=1)
print(classification_report(test_df_with_img['account_type_multi'], pred, digits=5))

52/52 [==============================] - 1s 13ms/step
              precision    recall  f1-score   support

           0    0.36973   0.46845   0.41328       412
           1    0.44712   0.52391   0.48248       460
           2    0.95045   0.77007   0.85081       274
           3    0.64644   0.47481   0.54749       516

    accuracy                        0.53550      1662
   macro avg    0.60344   0.55931   0.57351      1662
weighted avg    0.57280   0.53550   0.54623      1662

